In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:14pt;}
div.output {font-size:14pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:14pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:14pt;padding:5px;}
table.dataframe{font-size:14px;}
</style>
"""))

<b><font size="6" color="red">ch14_웹데이터 수집</font></b>
# 1절. BeautifulSoup과 parser
`pip install bs4` 아나나콘다를 설치하면 자동 설치되는 패키지 7500개에 포함

- 공식 Documentation : https://beautiful-soup-4.readthedocs.io/en/latest/
- pip  ~~ 최신버전이 설치되고 속도가 빠를다. 
- conda ~~ 최신버전 바로 이전 -안정적 설치

In [4]:
# 내 local PC의 파일을 url처럼 접근
import requests  # http 요청 처리하는 lib
from requests_file import FileAdapter # file://프로토콜을 다루기 위함

In [5]:
s = requests.Session()  # http 요청 처리하는 세션 객체
s.mount("file://", FileAdapter())
# file://경로로 들어오면 이 요청은 c:(로컬파일)을 접근
response = s.get('file:///ai/source/01_python/data/ch14_sample.html')
response

<Response [200]>

In [6]:
if response:
    print('잘 접근하였습니다.')
else:
    print('접근을 못했습니다.')

잘 접근하였습니다.


In [ ]:
response.status_code
    # 200 : 정상
    # 404 : 없는 페이지
    # 406 : get, post 오류

In [7]:
response.content   # 바이이너리 형식의 html내용

b'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90 \xec\x95\x88\xec\x9d\x98 \xeb\x82\xb4\xec\x9a\xa9</div>\r\n  <p>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\xb3\xec\x97\x90\xec\x84\x9c \xed\x99\x9c\xec\x9a\xa9\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4</p>\r\n  <div class="contents">\r\n    \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\xa5\xbc \xec\x96\xb4\xeb\x96\xbb\xea\xb2\x8c \xec\x9e\x91\xec\x84\xb1\xed\x95\x98\xeb\x8a\x90\xeb\x83\x90\xec\x97\x90 \xeb\x94\xb0\xeb\x9d\xbc\r\n    <span>\xeb\x8b\xa4\xeb\xa5\xb8<b>\xec\x9a\x94\xec\x86\x8c\xea\xb0\x80 \xeb\xb0\x98\xed\x99\x98</b></span>\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4\r\n  </div>\r\n  <div>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\

In [10]:
response.content.decode('utf-8')

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [12]:
response.text  # HTML파일의 텍스트 내용

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [13]:
# html 파싱
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content,   # respose.text
                    "html.parser")
# soup

In [20]:
# soup.select_one('선택자') : 해당선택자 처음 하나만
el = soup.select_one('h1')  # 처음 나오는 h1태그 하나만
print('el:',el)
print('el.text:',el.text)
print('el.string:',el.string)
print('el의 속성들:',el.attrs)
print('el의 id속성:',el.attrs['id'])  # el.attrs는 딕셔너리
print('el의 id속성:',el.attrs.get('id')) 
print('el의 href속성:',el.attrs.get('href'))  # el.attrs['href'] 하면 error
print('el의 태그이름 :', el.name)

el: <h1 class="greeting css" id="text">Hello, CSS</h1>
el.text: Hello, CSS
el.string: Hello, CSS
el의 속성들: {'class': ['greeting', 'css'], 'id': 'text'}
el의 id속성: text
el의 id속성: text
el의 href속성: None
el의 태그이름 : h1


In [21]:
# soup.select('선택자') : 해당 선택자 모두
els = soup.select('h1')  # h1태그를 list
print('els:',els)
print('els의 text들:',[el.text for el in els])
print('els의 string들:', [el.stirng for el in els])
print('els의 속성들:', [el.attrs for el in els])
print('els의 class 속성들:', [el.attrs.get('class') for el in els])

els: [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
els의 text들: ['Hello, CSS', 'Hi, CSS']
els의 string들: [None, None]
els의 속성들: [{'class': ['greeting', 'css'], 'id': 'text'}, {'class': ['css']}]
els의 class 속성들: [['greeting', 'css'], ['css']]


In [25]:
# soup.find(태그, 속성) : soup.select_one('선택자')와 유사
print('select_one :',soup.select_one('h1.css'))
print('find:',soup.find('h1',{'class':'css'}) ) # soup.select_one('h1.css')
print('find:',soup.find('h1', class_='css') )
print()
print('select_one :', soup.select_one('h1#text'))
print('find :', soup.find('h1',{'id':'text'}))


select_one : <h1 class="greeting css" id="text">Hello, CSS</h1>
find: <h1 class="greeting css" id="text">Hello, CSS</h1>
find: <h1 class="greeting css" id="text">Hello, CSS</h1>

select_one : <h1 class="greeting css" id="text">Hello, CSS</h1>
find : <h1 class="greeting css" id="text">Hello, CSS</h1>


In [29]:
# soup.find_all(태그, 속성) : soup.select('선택자')와 유사
print('selec :', soup.select('h1.css'))
print('find_all :', soup.find_all('h1', class_='css'))
print('find_all :', soup.find_all('h1', {'class':'css'}))

selec : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]


In [34]:
print('selec :', soup.select('h1.css, span'))
print('find_all :', soup.find_all(['h1','span'], {'class':'css'}))
print('find_all :', soup.find_all(['h1','span'], class_='css')+\
                     soup.find_all('span'))

selec : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]


In [37]:

print('select(빈 list) :', soup.select('a.css'))
print('select_one(None) :', soup.select_one('a.css'))
print('find_all(빈 list) :', soup.find_all('a',{'class','css'}))
print('find(None) :', soup.find('a',{'class': 'css'}))

select(빈 list) : []
select_one(None) : None
find_all(빈 list) : []
find(None) : None


# 2절. 정적 웹 데이터 수집(정적 웹크롤링)
## 2.1 BeautifulSoup을 활용한 html 웹 데이터 수집
### 1) 환율정보 가져오기(네이버증권 -> 시장 지표)
- https://finance.naver.com/marketindex/

In [38]:
# 크롤링 허용범위는 사이트마다  ~/robots.txt에서 확인 [https://finance.naver.com/robots]
    #Allow : / - 사이트의 모든 경로(/)에 대한 크롤링 허용

In [46]:
# 방법1 
import requests 
from bs4 import BeautifulSoup
url = 'https://finance.naver.com/marketindex/'
response = requests.get(url)
# response.status_code
# response.text / response.content.decode('cp949')
soup = BeautifulSoup(response.text, "html.parser")

In [50]:
# 방법2
from urllib.request import urlopen
url = 'https://finance.naver.com/marketindex/'
response = urlopen(url)
response.status
# response.read() / response.read().decode('cp949')
soup = BeautifulSoup(response, "html.parser")

'\n<script language=javascript src="/template/head_js.naver?referer=info.finance.naver.com&menu=marketindex&submenu=market"></script>\n<script type="text/javascript" src="https://ssl.pstatic.net/imgstock/static.pc/20251105171339/js/info/jindo.min.ns.1.5.3.euckr.js"></script>\n<script type="text/javascript" src="https://ssl.pstatic.net/imgstock/static.pc/20251105171339/js/jindo.1.5.3.element-text-patch.js"></script>\n\n\n\t\n\t\n\t<div id="container" style="padding-bottom:0px;">\n\t\n\n\t\t\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\t<div class="market_include">\n\t<div class="market_data">\n\t\t<div class="market1">\n\t\t\t<div class="title">\n\t\t\t\t<h2 class="h_market1"><span>환전 고시 환율</span></h2>\n\t\t\t</div>\n\t\t\t<!-- data -->\n\t\t\t<div class="data">\n\t\t\t\t<ul class="data_lst" id="exchangeList">\n\t\t\t\t\n\t\t\t\t    \n\t\t\t\t         \n\t\t\t\t         \n\t\t\t\t         \n\t\t\t\t         \t\t\t\t         \t\t\t         \n\t\t\t\t        \n\t\t\t\t\t\n\t\t\t\

In [57]:
# div.head_info > span.value(find함수)
prices = []
headinfos = soup.find_all('div',class_='head_info')
for headinfo in headinfos:
    price = headinfo.find('span', class_='value')
    # print(price.text.replace(',',''))        
    prices.append(float(''.join(price.text.split(','))))
print(prices)

[1454.5, 945.25, 1681.18, 204.23, 153.42, 1.1564, 1.3161, 99.47, 59.75, 1698.27, 4009.8, 189159.55]


In [61]:
# div.head_info > span.value(select함수)
prices = soup.select('div.head_info > span.value')
print([float(p.text.replace(',','')) for p in prices])

[1454.5, 945.25, 1681.18, 204.23, 153.42, 1.1564, 1.3161, 99.47, 59.75, 1698.27, 4009.8, 189159.55]


In [66]:
titles = soup.select('h3.h_lst > span.blind')
for t in titles:
    print(t.text, end='\t')
# print([t.text +'\t' for t in titles])

미국 USD	일본 JPY(100엔)	유럽연합 EUR	중국 CNY	달러/일본 엔	유로/달러	영국 파운드/달러	달러인덱스	WTI	휘발유	국제 금	국내 금	

In [69]:
units = soup.select('div.head_info > span > span.blind')
units = [unit.text for unit in units]
units.insert(7,'') # 7번째에 '' 추가
units

['원', '원', '원', '원', '엔', '달러', '달러', '', '달러', '원', '달러', '원']

In [71]:
statuses = soup.select('div.head_info > span.blind')
for idx in range(len(statuses)):
    print(statuses[idx].text, end='\t')


하락	하락	하락	하락	상승	하락	상승	하락	상승	상승	상승	상승	

In [72]:
len(titles), len(prices), len(units), len(statuses)

(12, 12, 12, 12)

In [74]:
for idx in range(len(titles)):
    print("{} : {}{} - {}".format(titles[idx].text,
                                 prices[idx].text,
                                 units[idx],
                                 statuses[idx].text))

미국 USD : 1,454.50원 - 하락
일본 JPY(100엔) : 945.25원 - 하락
유럽연합 EUR : 1,681.18원 - 하락
중국 CNY : 204.23원 - 하락
달러/일본 엔 : 153.4200엔 - 상승
유로/달러 : 1.1564달러 - 하락
영국 파운드/달러 : 1.3161달러 - 상승
달러인덱스 : 99.4700 - 하락
WTI : 59.75달러 - 상승
휘발유 : 1698.27원 - 상승
국제 금 : 4009.8달러 - 상승
국내 금 : 189159.55원 - 상승


In [79]:
for title, price, unit, status in zip(titles, prices, units, statuses):
    print("{} : {}{} - {}".format(title.text,
                                 price.text,
                                 unit,
                                 status.text))

미국 USD : 1,454.50원 - 하락
일본 JPY(100엔) : 945.25원 - 하락
유럽연합 EUR : 1,681.18원 - 하락
중국 CNY : 204.23원 - 하락
달러/일본 엔 : 153.4200엔 - 상승
유로/달러 : 1.1564달러 - 하락
영국 파운드/달러 : 1.3161달러 - 상승
달러인덱스 : 99.4700 - 하락
WTI : 59.75달러 - 상승
휘발유 : 1698.27원 - 상승
국제 금 : 4009.8달러 - 상승
국내 금 : 189159.55원 - 상승


### 2) 이번주 로또 번호 출력
- https://dhlottery.co.kr/gameResult.do?method=byWin (google에 로또번호 당첨번호 검색)
```
    1197회(2025년 11월 08일 추첨)
    당첨번호 [1,5,7,26,28,43]
    보너스 30
```

In [120]:
# 방법1 
import requests 
from bs4 import BeautifulSoup
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
response = requests.get(url)
response  #.status_code
# response.text / response.content.decode('cp949')
soup = BeautifulSoup(response.text, "html.parser")

In [124]:
# 방법2
from urllib.request import urlopen
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
response = urlopen(url)
response.status
soup = BeautifulSoup(response, "html.parser")
# soup

In [125]:
# div.win_result h4 strong   1197회  / div.win_result h4 당첨결과
times = soup.select_one('div.win_result h4')
times.text

'1197회 당첨결과'

In [126]:
# div.win_result p.desc (2025년 11월 08일 추첨)
d_day = soup.select_one('div.win_result p.desc')
d_day.text

'(2025년 11월 08일 추첨)'

In [127]:
# div.num_wins p span.ball_645 lrg ball1
lotto_number_el = soup.select('div.num.win  span')
lotto_number = [int(el.text) for el in lotto_number_el]
bonus_number = int(soup.select_one('div.num.bonus > p > span').text)

In [129]:
times = soup.select_one('div.win_result strong').text # 1197회    #============> 강사풀이
date  = soup.select_one('div.win_result > p.desc').text
# date = soup.find('p', class_='desc').text
lotto_number_el = soup.select('div.num.win  span')
lotto_number = [int(el.text) for el in lotto_number_el]
bonus_number = int(soup.select_one('div.num.bonus > p > span').text)

In [128]:
print(times, date)                  #============> 강사풀이
print('당첨번호 ',lotto_number)
print('보 너 스 ', bonus_number)

<h4><strong>1197회</strong> 당첨결과</h4> (2025년 11월 08일 추첨)
당첨번호  [1, 5, 7, 26, 28, 43]
보 너 스  30


### 3) 다음 검색 리스트
```
no title             link
0  "한풀꺽인 비트코인  https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=비트코인"   
```

In [138]:
# 방법1 : 가능
import requests 
from bs4 import BeautifulSoup
word = '비트코인'
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q='+ word
print(url)
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# 방법2 : 불가
# from urllib.request import urlopen
# from urllib.parse import quote
# word = quote('비트코인')   #url에 한글이 포함된 경우 한글을 quote()로 변환
# url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q='+ word
# print(url)
# response = urlopen(url)
# soup = BeautifulSoup(response, "html.parser")

items_find_list = [] #검색한 결과를 담을 리스트(딕셔너리 list)
items_el = soup.select('div.item-title > strong.tit-g > a')
for idx, item in enumerate(items_el):
        #print(idx, item.text, item.attrs.get('href'))
        items_find_list.append({'no':idx,
                                'title': item.text,
                                'link': item.attrs.get('href')})
import pandas as pd
pd.DataFrame(items_find_list)


https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=비트코인


,no,title,link
0,0,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
1,1,비트코인 1억5500만원대 소폭 상승…트럼프 발언탓?,http://v.daum.net/v/20251110092814031
2,2,"""비트코인 21억 간다더니""...갑자기 말 바꿨다",http://v.daum.net/v/20251109105545139
3,3,"트럼프 돈 푼다...비트코인, 관세 배당금에 반등 [매일코인]",http://v.daum.net/v/20251110112101913
4,4,"비트코인, 트럼프 ‘2000달러 지급’ 발언에 10만4000달러선 [크립토브리핑]",http://v.daum.net/v/20251110103033108
5,5,"“비트코인 약세, 닷컴버블과 판박이”…큰손·장투족 매일 던진다",http://v.daum.net/v/20251110094507791
6,6,비트코인 '와르르' 내려오는데 '의외'의 전망,http://v.daum.net/v/20251107210115339
7,7,"[가상자산 나침반] ‘겹악재’ 비트코인, 반전 포인트 나올까",http://v.daum.net/v/20251109112746909
8,8,"美 셧다운 종료 기대…비트코인, 10만 5000달러 회복[코인브리핑]",http://v.daum.net/v/20251110111435519
9,9,"비트플래닛, 비트코인 200개 매입 완료…자금 조달부터 커스터디까지 끝내",http://v.daum.net/v/20251110081500340


In [140]:
items_find_list = [] #검색한 결과를 담을 리스트
items_el = soup.select('div.item-title > strong.tit-g > a')
for idx, item in enumerate(items_el):
        items_find_list.append([idx, item.text, item.attrs['href']])
pd.DataFrame(items_find_list, columns=['번호','제목','링크'])

,번호,제목,링크
0,0,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
1,1,비트코인 1억5500만원대 소폭 상승…트럼프 발언탓?,http://v.daum.net/v/20251110092814031
2,2,"""비트코인 21억 간다더니""...갑자기 말 바꿨다",http://v.daum.net/v/20251109105545139
3,3,"트럼프 돈 푼다...비트코인, 관세 배당금에 반등 [매일코인]",http://v.daum.net/v/20251110112101913
4,4,"비트코인, 트럼프 ‘2000달러 지급’ 발언에 10만4000달러선 [크립토브리핑]",http://v.daum.net/v/20251110103033108
5,5,"“비트코인 약세, 닷컴버블과 판박이”…큰손·장투족 매일 던진다",http://v.daum.net/v/20251110094507791
6,6,비트코인 '와르르' 내려오는데 '의외'의 전망,http://v.daum.net/v/20251107210115339
7,7,"[가상자산 나침반] ‘겹악재’ 비트코인, 반전 포인트 나올까",http://v.daum.net/v/20251109112746909
8,8,"美 셧다운 종료 기대…비트코인, 10만 5000달러 회복[코인브리핑]",http://v.daum.net/v/20251110111435519
9,9,"비트플래닛, 비트코인 200개 매입 완료…자금 조달부터 커스터디까지 끝내",http://v.daum.net/v/20251110081500340


In [146]:
items_find_list = [] #검색한 결과를 담을 리스트
# div.item-title > strong.tit-g > a
item_titles = soup.find_all('div', class_='item-title')
for idx, item in enumerate(item_titles):
    a = item.find('a')
    # print(idx, a.text, a.attrs['href'])
    # items_find_list.append([idx, a.text, a.attrs['href']])
    items_find_list.append({
        'no':idx,
        'title':a.text,
        'link': a.attrs['href'],
    })
pd.DataFrame(items_find_list)

,no,title,link
0,0,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
1,1,비트코인 1억5500만원대 소폭 상승…트럼프 발언탓?,http://v.daum.net/v/20251110092814031
2,2,"""비트코인 21억 간다더니""...갑자기 말 바꿨다",http://v.daum.net/v/20251109105545139
3,3,"트럼프 돈 푼다...비트코인, 관세 배당금에 반등 [매일코인]",http://v.daum.net/v/20251110112101913
4,4,"비트코인, 트럼프 ‘2000달러 지급’ 발언에 10만4000달러선 [크립토브리핑]",http://v.daum.net/v/20251110103033108
5,5,"“비트코인 약세, 닷컴버블과 판박이”…큰손·장투족 매일 던진다",http://v.daum.net/v/20251110094507791
6,6,비트코인 '와르르' 내려오는데 '의외'의 전망,http://v.daum.net/v/20251107210115339
7,7,"[가상자산 나침반] ‘겹악재’ 비트코인, 반전 포인트 나올까",http://v.daum.net/v/20251109112746909
8,8,"美 셧다운 종료 기대…비트코인, 10만 5000달러 회복[코인브리핑]",http://v.daum.net/v/20251110111435519
9,9,"비트플래닛, 비트코인 200개 매입 완료…자금 조달부터 커스터디까지 끝내",http://v.daum.net/v/20251110081500340


In [175]:
# 다음 뉴스 검색(원하는 키워드를 원하는 페이지를 가져오기)
import requests
from bs4 import BeautifulSoup
import pandas as pd
def collect_news_list(keyword, page):
    'keyword로 page에 다음 뉴스검색한 결과를 출력 -> list를 return'
    # url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q={keyword}&p={page}'
    # response = requests.get(url)
    url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8'
    params = {'q':keyword, 'p':page}
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    items_find_list = []  # 검색한 결과를 담는 리스트
    item_titles = soup.select('div.item-title > strong.tit-g > a')
    print(keyword)
    for idx, item in enumerate(item_titles):
        items_find_list.append([(page-1)*10+idx,item.text, item.attrs['href']])    
    return items_find_list

In [176]:
collect_news_list('AI',3)

AI


[[20,
  ' [리셋 코리아] AI 대전환기, 소버린AI 도전 피하지 말아야 ',
  'http://v.daum.net/v/20251110002138457'],
 [21,
  ' [AI픽] 과도한 AI 의존 어떻게 할까…2025년 디지털 포용 포럼 ',
  'http://v.daum.net/v/20251110153112268'],
 [22,
  ' “AI·인간 \'공동과학자\' 시대…R&D 속도, AI 활용 능력이 가른다" ',
  'http://v.daum.net/v/20251110050654976'],
 [23,
  ' ‘AI 허브’ 된 韓… 산업 대전환 본격화 [심층기획-엔비디아발 AI 혁명] ',
  'http://v.daum.net/v/20251110060356516'],
 [24, ' AI로 과제 낸 학생 징계해야 합니까? ', 'http://v.daum.net/v/20251110135100847'],
 [25,
  ' AI로 신약개발 가속… 생명연, ‘K-AI 신약개발 전임상·임상 모델개발 사업’ 참여 ',
  'http://v.daum.net/v/20251110143647077'],
 [26,
  ' [에듀플러스]“AI는 테슬라인데, 대학은 여전 종이지도 그리고 있다”…대학 평가 제도, AI 시대 못 따라가 ',
  'http://v.daum.net/v/20251110135538027'],
 [27,
  ' “기본 없는 AI 전환 추진 실패…운영개선 잘해야 AI 더 축적” ',
  'http://v.daum.net/v/20251110050415922'],
 [28, ' 코레일, ‘철도 AI 전문가’ 직원 양성 가속도 ', 'http://v.daum.net/v/20251110141739063'],
 [29,
  ' ‘피지컬 AI’ 추진 엔비디아, 반도체·제조강국 韓과 ‘윈윈 협력’ [심층기획-엔비디아발 AI 혁명] ',
  'http://v.daum.net/v/20251110060456524']]

In [184]:
import time
keywords = ['청바지','동대문']
result0 = []  # 청바지 1~3페이지까지 검색한 결과를 담을 list
result1 = []  # 동대문 1~3페이지까지 검색한 결과를 담을 list
pages = 3
for i, keyword in enumerate(keywords):
    print(f'==={i+1}번째 검색어 {keyword} 검색 결과 수집 중입니다===')
    for page in range(1, pages+1):
        if i==0:
            result0.extend(collect_news_list(keyword, page))
        elif i==1:
            result1.extend(collect_news_list(keyword, page))
        time.sleep(0.5)

===1번째 검색어 청바지 검색 결과 수집 중입니다===
청바지
청바지
청바지
===2번째 검색어 동대문 검색 결과 수집 중입니다===
동대문
동대문
동대문


In [189]:
result0_df = pd.DataFrame(result0, columns=['no','title','link'])
result1_df = pd.DataFrame(result1, columns=['no','title','link'])
result1_df.head()

,no,title,link
0,0,"“돈 벌어가세요” 동대문 일요시장, 겨울옷 핫플된 사연? [르포]",http://v.daum.net/v/20251110100357735
1,1,"동대문구, ‘11월 자전거교실’ 개강… 올해 마지막 초보자 교육 진행",http://v.daum.net/v/20251110145114101
2,2,"음주운전 차, 동대문역서 인도 돌진...日 관광객 '사망'",http://v.daum.net/v/20251103073316480
3,3,"동대문구, 25일까지 '주민 건강도시학교'···""건강하게 살자""",http://v.daum.net/v/20251109223318356
4,4,"서울 동대문구, 15일 배봉산정원 가족축제 개최",http://v.daum.net/v/20251104104517122


In [192]:
result0_df.to_csv(f'data/ch14_{keywords[0]}.csv', index=False, encoding='cp949')
result1_df.to_csv(f'data/ch14_{keywords[1]}.csv', index=False)

### 4) User-Agent를 추가하여 크롤링
- 자동으로 브라우저를 통해 요청하는 것 처럼 보이기

In [199]:
# 방법2 :
from urllib.request import urlopen, Request
from urllib.parse import quote
word = quote('비트코인')  # 한글을 encoding 처리
url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&q='+word
print(url)
# headers = {'User-Agent':           
#            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
# request = Request(url, headers=headers)   # request객체 생성
request = Request(url)
request.add_header('User-Agent',           
                'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36')

response = urlopen(request)
soup = BeautifulSoup(response, 'html.parser')
# soup

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8


- https://www.melon.com/chart/index.htm
   * https://www.melon.com/robots.txt 에서 User-Agent에 봇이 지정

In [209]:
# 방법1
import requests
url = 'https://www.melon.com/chart/index.htm'
melonpage = requests.get(url)
melonpage     # <Response [406]> => 인증이 막혀있어 soup객체가 만들어지지 않는다.
# soup = BeautifulSoup(melonpage.text, 'html.parser')
# soup

<Response [406]>

In [213]:
# 방법2
from urllib.request import urlopen, Request
url = 'https://www.melon.com/chart/index.htm'
# melonpage = urlopen(url) # error
headers = {'user-agent':
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
request = Request(url, headers=headers)
melonpage = urlopen(request)
soup = BeautifulSoup(melonpage, 'html.parser')
# soup

In [223]:
# 방법1
url = 'https://www.melon.com/chart/index.htm'
headers = {'user-agent':
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
melonpage = requests.get(url, headers=headers)
soup = BeautifulSoup(melonpage.text, # melonpage.content
                    'html.parser')

ranks = soup.select('td div.wrap.t_center > span.rank')
titles = soup.select('div.ellipsis.rank01> span')  # [title.text.strip() for title in titles]
singers = soup.select('div.ellipsis.rank02')   # [singer.text.strip()[:20] for singer in singers]
# 1위 | 노래제목 - 가수명
for rank, title, singer in zip(ranks, titles, singers):
    print(f'{rank.text}위 | {title.text.strip()} - {singer.text.strip()[:20]}')

1위 | Blue Valentine - NMIXXNMIXX
2위 | 타임캡슐 - 다비치다비치
3위 | Golden - HUNTR/X, EJAE, AUDRE
4위 | Drowning - WOODZWOODZ
5위 | Good Goodbye - 화사 (HWASA)화사 (HWASA)
6위 | SPAGHETTI (feat. j-hope of BTS) - LE SSERAFIM (르세라핌), 
7위 | 달리 표현할 수 없어요 - 로이킴로이킴
8위 | 뛰어(JUMP) - BLACKPINKBLACKPINK
9위 | Soda Pop - KPop Demon Hunters C
10위 | 어제보다 슬픈 오늘 - 우디 (Woody)우디 (Woody)
11위 | FAMOUS - ALLDAY PROJECTALLDAY
12위 | 시작의 아이 ❍ - 박다혜, 마크툽 (MAKTUB)박다혜
13위 | IRIS OUT - Kenshi YonezuKenshi 
14위 | XOXZ - IVE (아이브)IVE (아이브)
15위 | 모르시나요(PROD.로코베리) - 조째즈조째즈
16위 | Rich Man - aespaaespa
17위 | Hollywood Action - BOYNEXTDOORBOYNEXTDO
18위 | 한번 더 이별 - 이창섭이창섭
19위 | 내게 사랑이 뭐냐고 물어본다면 - 로이킴로이킴
20위 | 너에게 닿기를 - 10CM10CM
21위 | HOME SWEET HOME (feat. 태양, 대성) - G-DRAGONG-DRAGON
22위 | 운명 (2025) - 먼데이 키즈, 이이경먼데이 키즈, 이
23위 | body - 다영 (DAYOUNG)다영 (DAYO
24위 | toxic till the end - 로제 (ROSÉ)로제 (ROSÉ)
25위 | Whiplash - aespaaespa
26위 | 시작의 아이 - 마크툽 (MAKTUB)마크툽 (MAK
27위 | 천상연 - 이창섭이창섭
28위 | 나는 반딧불 - 황가람황가람
29위 | 어떻게 이별까지 사랑하겠어, 널 사랑하는 거지 - AK

### 5) 네이버 지식인으로 검색(open API 사용X)
- 특정 keyword를 특정페이지 수 만큼

In [5]:
# 방법1
from requests import get
from bs4 import BeautifulSoup
keyword = '쳇지피티'
# url = f'https://kin.naver.com/search/list.naver' # ?query={keyword}'
# # print(url)
# params = {'query': keyword}
# response = get(url, params=params)

url = f'https://kin.naver.com/search/list.naver?query={keyword}'   # 파라미터가 적은 경우 유용
response = get(url)
print(response.status_code)
soup = BeautifulSoup(response.text,'html.parser')
# soup

https://kin.naver.com/search/list.naver?query=쳇지피티
200


In [10]:
# 방법2  #url에 한글이 포함되어 있으면 인코딩해서 처리하는 방식
from urllib.request import urlopen
from urllib.parse import quote
keyword = quote('쳇지피티')
url = f'https://kin.naver.com/search/list.naver?query={keyword}'  
print(url)
response = urlopen(url)
print(response.status)
soup = BeautifulSoup(response, 'html.parser')
# soup

https://kin.naver.com/search/list.naver?query=%EC%B3%87%EC%A7%80%ED%94%BC%ED%8B%B0
200


In [22]:
# 페이징 포함
from urllib.request import urlopen
from urllib.parse import quote
from bs4 import BeautifulSoup
import pandas as pd
keyword = quote('쳇지피티')
pages = 3
items_list = []   # 크롤링한 데이터를 담을 list(title, link)
for page in range(1, pages+1):
    url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
    response = urlopen(url)
    # print(response.status)
    soup = BeautifulSoup(response, 'html.parser')
    items = soup.select('dt > a')
    # print(len(items))# 10
    for item in items:
        title = item.text
        link  = item.attrs.get('href')
        # print(title, link)
        items_list.append({
            'title': title,
            'link' : link
        })
df = pd.DataFrame(items_list)
print(df.shape)
print(df.loc[29,'link'])
df.sample()

(30, 2)
https://kin.naver.com/qna/detail.naver?d1id=3&dirId=31301&docId=485933293&qb=7LOH7KeA7ZS87Yuw&enc=utf8


,title,link
4,쳇지피티 3일 리셋,https://kin.naver.com/qna/detail.naver?d1id=8&...


## 2.2 openAPI사용 : json 웹데이터 수집
### 1) 네이버 지식인으로 검색(open API 사용O)
- 네이버 개발자센터에서 애플리케이션등록(이름/검색/WEB설정 http://localhost)
- .env파일에 CLIENT_ID/CLIENT_SECRET 환경변수 저장
- 환경변수를 읽기 위해서 `pip install dotenv`
- 특정 keyword를 지식인 검색(데이터수 30개)

In [23]:
%pip install dotenv

Note: you may need to restart the kernel to use updated packages.


In [26]:
# 환경변수 읽어오기
from dotenv import load_dotenv
import os
load_dotenv(
    #dotenv_path='.env'
            )  # 현 소스와 같은 폴더내의  .env를 메모리에 load
# print(os.getenv('CLIENT_ID'))
# print(os.getenv('CLIENT_SECRET'))

True

In [32]:
# 네이버 검색 API 예제 - 블로그 검색  
## 방법2
import os
import urllib.request
import json
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
encText = urllib.parse.quote("쳇지피티")
url = "https://openapi.naver.com/v1/search/blog?query=" + encText # JSON 결과

# request = urllib.request.Request(url)
# request.add_header("X-Naver-Client-Id",client_id)
# request.add_header("X-Naver-Client-Secret",client_secret)
headers = {
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret
}
request = urllib.request.Request(url, headers=headers)

response = urllib.request.urlopen(request)

rescode = response.status
if(rescode==200):
    response_body = response.read()   # json
    print(type(response_body.decode('utf-8')))  # 딕셔너리가 아니라 문자열
    print(response_body.decode('utf-8')[:50])   # 딕셔너리로 변환 필요
    
else:
    print("Error Code:" + rescode)

<class 'str'>
{
	"lastBuildDate":"Tue, 11 Nov 2025 11:48:00 +090


In [45]:
# 방법1
import requests
import pandas as pd
import json  # response 텍스트를 json스타일의 딕셔너리로
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
query = '쳇지피티'
headers = {
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret
}
# url = f'https://openapi.naver.com/v1/search/kin?query={query}&display=30'
# response = requests.get(url, headers=headers)
url = 'https://openapi.naver.com/v1/search/kin'
params = {'query':query, 'display':30}
response = requests.get(url, params=params, headers=headers)
# print(response.text[:500])
# items = json.loads(response.text)['items']
items = response.json()['items']
items_list = []
for item in items:
    title = item['title'].replace('<b>','').replace('</b>','')
    link  = item.get('link')
    description = item.get('description').replace('<b>','').replace('</b>','')
    items_list.append([title, link, description])
pd.DataFrame(items_list, columns=['title','link','description']).head()

,title,link,description
0,쳇지피티 사주 정확성,https://kin.naver.com/qna/detail.naver?d1id=3&...,쳇지피티 사주 정확성 몇프로 정도 될까요? 큰틀은 얼추 맞는거 같기도하고 모르겠네요...
1,쳇지피티 관해서,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티한테 질문 제대로 하려면 어떻게 해야 하냐요?... 쳇지피티 한테 하는질문도...
2,사주에서 뭐가 부족한지 봐주세요(쳇지피티 답변 금지),https://kin.naver.com/qna/detail.naver?d1id=3&...,사주에서 뭐가 부족한지 봐주세요(쳇지피티 답변 금지) 남매입니다 남자 1986년 9...
3,쳇지피티 유료로 전환시,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티 무료버전으로 카톡대화창 전송해서 상대의심리에 관한 분석요청을 많이했는데 첨...
4,쳇지피티 3일 리셋,https://kin.naver.com/qna/detail.naver?d1id=8&...,... 쳇지피티 어떻게 하는건가욤 유료결제인가요 유료결제는 얼마인가요 그거 하게되면...


### quiz) 네이버 openAPI를 이용해서 청바지 이미지 100건의 데이터를 ch14_청바지.csv출력
```
제목, 링크, 썸네일, sizeheight, sizewidth    
```

In [63]:
def get_image_list(query):
    'query로 검색한 이미지 정보(정보,링크,썸네일,크기) 100건 데이터 프레임을 return(방법1)' # requests.get(url, params=params, headers=headers)
    import requests
    import json
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    url = 'https://openapi.naver.com/v1/search/image'
    params = {'query':query, 'display':100 }
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    response = requests.get(url, params=params, headers=headers)
    #items = response.json()['items']
    items = json.loads(response.text)['items']
    print(items[0])
    items_list = []
    for item in items:
        #print(item)
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        items_list.append({
            '제목':item.get('title'),
            '링크':link,
            '썸네일': thumbnail,
            'sizeheight': int(item.get('sizeheight')),
            'sizewidth':  int(item.get('sizewidth')),
        })
    return pd.DataFrame(items_list)
df = get_image_list("청바지")
df.to_csv('data/ch14_청바지.csv')
df.sample()

{'title': '여성 하이웨이스트 연청 와이드 청바지 통바지 편안함 스타일리시 캐주얼 데님 : 믿음상점 스토어', 'link': 'https://shop-phinf.pstatic.net/20251015_281/1760512390478oRzer_PNG/42211957144911892_1321802660.png', 'thumbnail': 'https://search.pstatic.net/common/?type=b150&src=https://shop-phinf.pstatic.net/20251015_281/1760512390478oRzer_PNG/42211957144911892_1321802660.png', 'sizeheight': '1000', 'sizewidth': '1000'}


,제목,링크,썸네일,sizeheight,sizewidth
72,NJKLJ013 빅 사이즈 봄가을 데님팬츠 청바지1+1 : 키작녀의류 신발전문,http://shop1.phinf.naver.net/20251020_220/1760...,https://search.pstatic.net/common/?type=b150&s...,1200,1200


In [60]:
def get_image_list(query):
    'query로 검색한 이미지 정보(정보,링크,썸네일,크기) 100건 데이터 프레임을 return(방법2)'  #urlopen(request)
    from urllib.request import urlopen, Request
    from urllib.parse import quote
    import json
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    encText = quote(query)
    url = f'https://openapi.naver.com/v1/search/image?query={encText}&display=100'
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    request = Request(url, headers=headers)
    response = urlopen(request)
    # print(response.read().decode('utf-8'))
    items = json.loads(response.read().decode('utf-8'))['items']
    items_list = []
    for item in items:
        #print(item)
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        items_list.append({
            '제목':item.get('title'),
            '링크':link,
            '썸네일': thumbnail,
            'sizeheight': int(item.get('sizeheight')),
            'sizewidth':  int(item.get('sizewidth')),
        })
        
    return pd.DataFrame(items_list)
df = get_image_list("청바지")
df.to_csv('data/ch14_청바지.csv')
df.sample()

,제목,링크,썸네일,sizeheight,sizewidth
78,남성와이드청바지 남자일자청바지 기모청바지 : 멋있다,http://shop1.phinf.naver.net/20231127_260/1701...,https://search.pstatic.net/common/?type=b150&s...,1000,1000


In [59]:
print(df.loc[99, '링크'])
print(df.loc[99, '썸네일'])

http://shop1.phinf.naver.net/20221221_230/1671613475090ETHL3_JPEG/72749320788507671_1995218513.jpg
https://search.pstatic.net/common/?type=b150&src=http://shop1.phinf.naver.net/20221221_230/1671613475090ETHL3_JPEG/72749320788507671_1995218513.jpg


In [71]:
def save_image(attr, idx, link, query):
    'link의 이미지를 image/attr_idx_query.확장자로 local에 저장'
    import requests
    file_extension = link.split('.')[-1]  # 확장자
    quote_index = file_extension.find('?')  # 확장자뒤에 ?가 있는 위치 .jpg?w=780
    if quote_index != -1:
        file_extension = file_extension[:quote_index]
    img = requests.get(link).content  # 바이너리로 읽는다.
    with open(f'image/{attr}_{idx+1}_{query}.{file_extension}','wb') as f:
        f.write(img)
save_image('메인', 0, df.loc[25, '링크'], '청바지')

온라인상의 이미지는 DL/ML에서 학습이 안되므로 내 PC에 저장 후 학습시키기위해서...

In [72]:
def get_image_list_save_image(query):
    'query로 검색한 이미지 정보(정보,링크,썸네일,크기) 100건 데이터 프레임을 return(방법1)' # requests.get(url, params=params, headers=headers)
    import requests
    import json
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    url = 'https://openapi.naver.com/v1/search/image'
    params = {'query':query, 'display':100 }
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    response = requests.get(url, params=params, headers=headers)
    #items = response.json()['items']
    items = json.loads(response.text)['items']
    items_list = []   # 정보가 담길 리스트
    for idx, item in enumerate(items):
        
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        items_list.append({
            '제목':item.get('title'),
            '링크':link,
            '썸네일': thumbnail,
            'sizeheight': int(item.get('sizeheight')),
            'sizewidth':  int(item.get('sizewidth')),
        })
        # 이미지 파일 저장
        save_image('메인',idx,link,query)
        save_image('썸네일',idx,thumbnail,query)
        if (idx%20 == 0) & (idx!=0):
            print(f'= = = {idx}% 진행완료 = = =')
    print('= = = 완료 = = =')
    return pd.DataFrame(items_list)
df = get_image_list_save_image("청바지")
df.to_csv('data/ch14_청바지.csv')
df.sample()

= = = 20% 진행완료 = = =
= = = 40% 진행완료 = = =
= = = 60% 진행완료 = = =
= = = 80% 진행완료 = = =
= = = 완료 = = =


,제목,링크,썸네일,sizeheight,sizewidth
22,남성 OB 통넓은 와이드핏 청바지 IC-JEA-RM517 : 파파브로맨,https://shop-phinf.pstatic.net/20250918_174/17...,https://search.pstatic.net/common/?type=b150&s...,1000,1000


## 2.3 XML 웹데이터 수집
- RSS / open API 을 통한 XML웹데이터 수집

### 1) 전국 날씨 RSS를 BeautifulSoup을 이용한 크롤링
- 구글에 "기상청 RSS" 검색한 첫번째 사이트로 이동

In [95]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
items_list = []
url = 'https://www.kma.go.kr/repositary/xml/fct/mon/img/fct_mon1rss_108_20251106.xml'
# 방법1
# target = requests.get(url)
# soup = BeautifulSoup(target.text, "xml")  # pip install lxml 필요없음
# 방법2
target = urlopen(url)
soup = BeautifulSoup(target, "xml")
local_tas = soup.select('local_ta')
# len(local_tas)
for local_ta in local_tas:
    local_name = local_ta.select_one('local_ta_name').text
    week1_normalYear = local_ta.select_one('week1_local_ta_normalYear').text
    week1_similarRange = local_ta.select_one('week1_local_ta_similarRange').text
    week1_minVal = local_ta.select_one('week1_local_ta_minVal').text
    week1_similarVal = local_ta.select_one('week1_local_ta_similarVal').text
    week1_maxVal = local_ta.select_one('week1_local_ta_maxVal').text
    items_list.append({
        '지역': local_name,
        '1주평년기온': week1_normalYear,
        '1주기온범위': week1_similarRange,
        '1주낮을확률': week1_minVal,
        '1주비슷할확률':week1_similarVal,
        '1주높을확률':week1_maxVal
    })
df = pd.DataFrame(items_list)
df['1주평년기온'] = df['1주평년기온'].astype(np.float64)
df['1주낮을확률'] = df['1주낮을확률'].astype(np.int16)
df['1주비슷할확률'] = df['1주비슷할확률'].astype(np.int16)
df['1주높을확률'] = df['1주높을확률'].astype(np.int16)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   지역       13 non-null     object 
 1   1주평년기온   13 non-null     float64
 2   1주기온범위   13 non-null     object 
 3   1주낮을확률   13 non-null     int16  
 4   1주비슷할확률  13 non-null     int16  
 5   1주높을확률   13 non-null     int16  
dtypes: float64(1), int16(3), object(2)
memory usage: 518.0+ bytes


### 2) XML응답하는 open API활용
- data.go.kr에서
    - 서울특별시_노선정보조회 서비스(버스ID, 정류장목록과 정류장ID)
    - 서울특별시_버스위치정보조회 서비스(실시간 버스 위치 목록)

In [ ]:
# STEP 1 버스번호의 버스id 받아오기
# 서울특별시_노선정보조회 서비스 - 3번 기능(getBusRouteList) 이용

In [10]:
from dotenv import load_dotenv
import os
load_dotenv()
# print(os.getenv('KEY'))  ## 확인 후 깃에 올라가지 않도록 막아둘 것 !!꼭
# print(os.getenv('KEY1'))

True

In [3]:
from bs4 import BeautifulSoup
# urlretrieve(url, 저장경로) : url의 파일을 저장경로에 저장
from urllib.request import urlretrieve
from urllib.parse import quote
# busNum = quote('구로09')
busNum = '162'
key = os.getenv('KEY')
url = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey={key}&strSrch={busNum}'
print(url)
savefilename1 = 'data/ch14_busInfo.xml'
urlretrieve(url, savefilename1)  # xml파일(url)을 savefilename으로 저장
with open(savefilename1, encoding='utf-8') as f:
    xml = f.read();
soup = BeautifulSoup(xml, 'xml')

http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey=zOP%2FMN6fuxug2JFL1ZRlDlYWPvCsAK9GsgTot%2FUyi5L94F3Coh1MFRPmtFH4gkISul%2B3VzIg5eRO5WWTgErzVw%3D%3D&strSrch=162


In [11]:
# 버스 노선 ID 가져오기
import requests
from urllib.parse import quote
busNum = '162'
key = os.getenv('KEY')
url1 = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey={key}&strSrch={busNum}'
response = requests.get(url1)
soup = BeautifulSoup(response.text, 'xml')

for item in soup.select('itemList'):
    busRouteNm = item.select_one('busRouteNm').text
    if busRouteNm == busNum:
        busRouteId = item.select_one('busRouteId').text
        break;
print('busRouteId =', busRouteId)

busRouteId = 100100034


In [12]:
# step2. 버스id로 버스정류장목록방아오기(정류장명, 정류장id, 경도, 위도)
# 서울특별시_노선정보 조회 서비스 -4번 기능(getStaionsByRouteList)

In [19]:
pd.options.display.max_rows = 60  # 출력 최대 행수 지정 default=60

In [21]:
import pandas as pd
url2 = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey={key}&busRouteId={busRouteId}'
response = requests.get(url2)
soup = BeautifulSoup(response.content, 'xml')
itemLists = soup.find_all('itemList')
print(f'{busNum}번 정류장 갯수: {len(itemLists)}')

bus_station = []
for itemList in itemLists:
    stationNm = itemList.find('stationNm').text  # 정류소 이름
    station   = itemList.find('station').text    # 정류소 고유 ID
    gpsX      = itemList.find('gpsX').text       # 경도
    gpsY      = itemList.find('gpsY').text       # 위도
    bus_station.append([stationNm, station, gpsX, gpsY])
df_station = pd.DataFrame(bus_station, columns=['정류소명','id','경도','위도'])
df_station.head()

162번 정류장 갯수: 77


,정류소명,id,경도,위도
0,정릉산장아파트,107000071,127.003343,37.616712
1,정릉4동주민센터.경국사,107000073,127.006345,37.613529
2,북한산보국문역2번출구,107000518,127.0079858233,37.612293899
3,성북청수도서관.정릉4동성당,107000075,127.0084193769,37.6115696748
4,정릉시장입구,107000077,127.0098212542,37.6084653256


In [7]:
# step3. 버스id로 실시간 위치정보를 받아오기(차량번호, 혼잡도, 최종정류장id, 다음정류장id, 도착소요시간)
# 서울특별시_버스위치정보조회 서비스 - 2번(getBusPosByRtidList)

In [81]:
url3 = f'http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?serviceKey={key}&busRouteId={busRouteId}'
print(url3)
response = requests.get(url3)
soup = BeautifulSoup(response.text, 'xml')
itemLists = soup.select('itemList')
print(f'운행중인 버스는 {len(itemList)}대입니다.')

bus_position = []
for itemList in itemLists:
    plainNo = itemList.select_one('plainNo').text  # 차량번호
    congetion = itemList.select_one('congetion').text  # 혼잡도(congetion)
    # 0:없음, 3:여유, 4:보통, 5:혼잡, 6:매우혼잡
    congetion = '없음' if congetion=='0' \
           else '여유' if congetion=='3' \
           else '보통' if congetion=='4' \
           else '혼잡' if congetion=='5' else '매우혼잡'
    gpsX = itemList.select_one('gpsX').text  # 경도
    gpsY = itemList.select_one('gpsY').text  # 위도
    lastStnId = itemList.select_one('lastStnId').text  # 최종정류소ID
    nextStId = itemList.select_one('nextStId').text  # 다음정류소ID
    nextStTm = itemList.select_one('nextStTm').text  # 다음정류소까지 소요시간
    
    bus_position.append({
        '차량번호':plainNo,
        '혼잡도':congetion,
        '경도':gpsX,
        '위도':gpsY,
        '최종정류소ID':lastStnId,
        '다음정류소ID':nextStId,
        '도착소요시간':nextStTm
    })
df_position = pd.DataFrame(bus_position)
df_position.head(2)

http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?serviceKey=zOP%2FMN6fuxug2JFL1ZRlDlYWPvCsAK9GsgTot%2FUyi5L94F3Coh1MFRPmtFH4gkISul%2B3VzIg5eRO5WWTgErzVw%3D%3D&busRouteId=100100034
운행중인 버스는 23대입니다.


,차량번호,혼잡도,경도,위도,최종정류소ID,다음정류소ID,도착소요시간
0,서울74사2216,여유,127.011512,37.605089,107000079,107000168,160
1,서울74사2218,보통,127.015284,37.596453,107000172,101000042,1528


In [82]:
df_station.loc[df_station['id']=='107000079','정류소명'].iloc[0]

'정릉우체국앞'

In [83]:
df_station.loc[df_station['id']=='107000071','정류소명'].iloc[0]

'정릉산장아파트'

In [84]:
def station_name(row):
    row = row.copy()
    'row의 이전정류소명과 다음정류소명을 추가하고, 도착소요시간을 분으로 바꿔 return'
    row['이전정류소명'] = df_station.loc[df_station['id']==row['최종정류소ID'],'정류소명'].iloc[0]
    row['다음정류소명'] = df_station.loc[df_station['id']==row['다음정류소ID'],'정류소명'].iloc[0]
    minite = int(row['도착소요시간'])// 60  # // 몫연산자
    sec    = int(row['도착소요시간']) % 60
    row['도착소요시간']  = f'{minite}분{sec}초'    
    return row
# station_name(df_position.loc[20])

In [86]:
df = df_position.apply(station_name, axis=1)
df.drop(['최종정류소ID','다음정류소ID'], axis=1).head()

,차량번호,혼잡도,경도,위도,도착소요시간,이전정류소명,다음정류소명
0,서울74사2216,여유,127.011512,37.605089,2분40초,정릉우체국앞,정릉입구.정릉역
1,서울74사2218,보통,127.015284,37.596453,25분28초,돈암2동주민센터입구.흥천사,해운센터.롯데영플라자
2,서울70사6558,여유,126.989779,37.577424,14분2초,창경궁.서울대학교병원,해운센터.롯데영플라자
3,서울74사1638,여유,126.982826,37.572486,5분16초,조계사.종로경찰서,해운센터.롯데영플라자
4,서울74사1536,여유,126.981461,37.562476,12분46초,해운센터.롯데영플라자,남영역.민주화운동기념관


# 3절 연습문제(Quiz1)
- yes24 의 베스트셀러 정보를 제공하는 사이트에서 베스트셀러 정보를 수집해서 파일에 저장하세요
    * 베스트셀러 정보 수집 주소 : http://www.yes24.com/24/category/bestseller (1페이지만 추출)
    * https://www.yes24.com/product/category/bestseller?pageNumber={page}
    ```
    ch14_yes24.csv 나 ch14_yes24의 내용(구분문자를 ,나 \t,....)
        - 순위,책이름,저자및출판사,가격
        1,트렌드코리아2026,"김난도,전미영,최지혜,권정윤,한다혜 | 미래의창",18,000원
        24,그 비스트 어쩌구,후쿠다신이치글그림/박연지역 | 소미미디어,25,200원       
    ```

In [108]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd

pageNumber = 1
url = f'https://www.yes24.com/product/category/bestseller?pageNumber={pageNumber}'
response = requests.get(url)
# print(url)
soup = BeautifulSoup(response.text, "html.parser")
# soup 
books = soup.select('div.itemUnit')
# books
ranks, titles, authors, publishes, prices = [], [], [], [], []
for book in books:
    rank   = book.select_one('em.ico.rank')
    title  = book.select_one('a.gd_name')
    author = book.select_one('span.authPub.info_auth')
    publish= book.select_one('span.authPub.info_pub > a')
    price  = book.select_one('strong.txt_num')

    ranks.append(rank.get_text(strip=True) if rank else '')          # 순위    em.ico.rank  
    titles.append(title.get_text(strip=True) if title else '')       # 책이름  a.gd_name   
    authors.append(author.get_text(strip=True) if author else '')    # 저자    span.authPub.info_auth 
    publishes.append(publish.get_text(strip=True) if publish else '') # 출판사  span.authPub.info_pub > a 
    prices.append(price.get_text(strip=True) if price else '')       # 가격    strongtxt_num

rows = list(zip(ranks, titles, authors, publishes, prices))
df = pd.DataFrame(rows, columns=['순위','제목','저자','출판사','가격'])
df.to_csv('data/ch14_yes24.csv', index=False, encoding='utf-8')

print("data/ch14_yes24.csv 파일로 저장 완료!")
print(df.head())

data/ch14_yes24.csv 파일로 저장 완료!
  순위             제목                                                 저자    출판사  \
0  1   트렌드 코리아 2026  김난도,전미영,최지혜,권정윤,한다혜저 외 7명정보 더 보기/감추기김난도전미영최지혜권...   미래의창   
1  2  처음 만나는 양자의 세계                                               채은미저   북플레저   
2  3  대형주 추세추종 투자법칙                                           이종호(전황)저  사피엔테스   
3  4           위버멘쉬                                          프리드리히 니체저    떠오름   
4  5   박곰희 연금 부자 수업                                               박곰희저  인플루엔셜   

        가격  
0  18,000원  
1  17,100원  
2  19,800원  
3  16,020원  
4  18,900원  


In [109]:
# import               ===================강사 풀이========================
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re

In [111]:
# 방법1
pages = 2
bestseller_list = []
with open('data/ch14_yes24.txt', 'w', encoding='utf-8') as f:  # 빈 파일만들기
    pass
for page in range(1, pages+1):
    url = f'https://www.yes24.com/product/category/bestseller?pageNumber={page}'
    #print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # 순위
    ranks_els = soup.select('div.img_upper > em.ico.rank')
    ranks =  [int(ranks_el.text) for ranks_el in ranks_els]
    # 책제목
    titles_els=soup.select("div.info_row > a.gd_name")
    titles = [titles_el.text for titles_el in titles_els]
    # 저자
    authors_els = soup.select('span.authPub.info_auth')
    authors = [authors_el.text.strip() for authors_el in authors_els]
    for i, author in enumerate(authors):
        match = re.search(r"(.*)외 \d+명", author)
        if(match):
            authors[i] = match.group()
            print(i, '번째 도서의 저자가 많아 외 x명 뒤에도 길어 그 앞까지 짜름')
    # 출판사
    publishers_els = soup.select('div.info_row.info_pubGrp > span.authPub.info_pub')
    publishers = [publishers_el.text for publishers_el in publishers_els]
    # 가격
    prices_els = soup.select("div.info_row > strong.txt_num")
    prices = [prices_el.text for prices_el in prices_els]
    
    # print(len(ranks), len(titles), len(writers), len(publishers), len(prices))
    with open('data/ch14_yes24.txt', 'a', encoding='utf-8') as f:
        for rank, title, author, publisher, price in zip(ranks, titles, authors, publishers, prices):
            # print("{:02},{},{} | {},{}".format(rank, title, author, publisher, price))
            f.write(f'{rank}, {title}, {author} | {publisher}, {price}\n')
            bestseller_list.append([rank, title, author, publisher, price])
df = pd.DataFrame(bestseller_list, columns=['순위','책이름','저자','출판사','가격'])
df.to_csv('data/ch14_yes24.csv', index=False)
df.head(1)

0 번째 도서의 저자가 많아 외 x명 뒤에도 길어 그 앞까지 짜름
10 번째 도서의 저자가 많아 외 x명 뒤에도 길어 그 앞까지 짜름


,순위,책이름,저자,출판사,가격
0,1,트렌드 코리아 2026,"김난도, 전미영, 최지혜, 권정윤, 한다혜 저 외 7명",미래의창,"18,000원"


In [ ]:
# 방법2
pages = 2
bestseller_list = []
with open('data/ch14_yes24.txt', 'w', encoding='utf-8') as f:
    pass
for page in range(1, pages+1):
    url = f'https://www.yes24.com/product/category/bestseller?pageNumber={page}'
    #print(url)
    response = urlopen(url)
    soup = BeautifulSoup(response, 'html.parser')
    # 순위
    ranks_els = soup.select('div.img_upper > em.ico.rank')
    ranks =  [int(ranks_el.text) for ranks_el in ranks_els]
    # 책제목
    titles_els=soup.select("div.info_row > a.gd_name")
    titles = [titles_el.text for titles_el in titles_els]
    # 저자
    authors_els = soup.select('span.authPub.info_auth')
    authors = [authors_el.text.split('정보 더 보기/감추기')[0].strip() 
                                   for authors_el in authors_els]
#     for i, author in enumerate(authors):
#         match = re.search(r"(.*)외 \d+명", author)
#         if(match):
#             authors[i] = match.group()
#             print(i, '번째 도서의 저자가 많아 외 x명 뒤에도 길어 그 앞까지 짜름')
    # 출판사
    publishers_els = soup.select('div.info_row.info_pubGrp > span.authPub.info_pub')
    publishers = [publishers_el.text for publishers_el in publishers_els]
    # 가격
    prices_els = soup.select("div.info_row > strong.txt_num")
    prices = [prices_el.text for prices_el in prices_els]
    # print(len(ranks), len(titles), len(writers), len(publishers), len(prices))
    with open('data/ch14_yes24.txt', 'a', encoding='utf-8') as f:
        for rank, title, author, publisher, price in zip(ranks, titles, authors, publishers, prices):
            # print("{:02},{},{} | {},{}".format(rank, title, author, publisher, price))
            f.write(f'{rank}, {title}, {author} | {publisher}, {price}\n')
            bestseller_list.append([rank, title, author, publisher, price])
df = pd.DataFrame(bestseller_list, columns=['순위','책이름','저자','출판사','가격'])
df.to_csv('data/ch14_yes24.csv', index=False)
df.head(1)